In [10]:
import numpy as np
import math
from IPython.core.debugger import set_trace

lowerBound, upperBound = -30, 30
x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
N = len(x)


def f(x):
    #Sphere
    # x in interval [-30, -30]
    #Sum = 0
    #for i in range(0,N):
    #    Sum = Sum + (x[i])**2

    #Rosenbrock
    #za epsilon = 10e-30 dao nulu
    # x in interval [-30, -30]
    Sum = 0
    for i in range(0, N-1):
        Sum = Sum + 100 * ((x[i])**2 - x[i+1])**2 + (x[i] - 1)**2
        
    #Ackley
    # x in interval [-5, -10]
    #Sum1, Sum2 = 0, 0
    #for i in range(0,N):
    #    Sum1 = Sum1 + (x[i])**2
    #    Sum2 = Sum2 + math.cos(2*math.pi*x[i])
    #Sum= -20.0*math.exp(-0.2*math.sqrt((1/N)*(Sum1)))-math.exp((1/N) * Sum2) + math.e + 20
    return Sum

l = [lowerBound for i in range(10)]
u = [upperBound for i in range(10)]

In [11]:
def _e(index,size):
    arr = np.zeros(size)
    arr[index] = 1.0
    return arr

In [12]:
def Delta(x):
    Sum = 0
    #no general, only box constraints
    for i in range(0,N):
        Sum = Sum + max(x[i]-u[i],0) + max(l[i]-x[i], 0)
    return Sum

In [13]:
def better(x,y):
    Dx = Delta(x)
    Dy = Delta(y)
    if(Dx < Dy):
        return True
    elif(Dx == Dy and f(x) < f(y)):
        return True
    else:
        return False

In [14]:
def findExtremes(s):
    Ds = np.empty([N+1])
    fs = np.empty([N+1])
    
    for i in range(0, N+1):
        Ds[i] = Delta(s[i])
        fs[i] = f(s[i])
    
    arg_b = np.argmin(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_b] and i != arg_b:
            if(fs[i] < fs[arg_b]):
                arg_b = i;
                
    arg_w = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_w] and i != arg_w:
            if(fs[i] > fs[arg_w]):
                arg_w = i;
    
    #mask the worst to find the second worst
    Ds[arg_w] = 0
    
    arg_sw = np.argmax(Ds)
    for i in range(0, N+1):
        if Ds[i] == Ds[arg_sw] and i != arg_w:
            if(fs[i] > fs[arg_sw]):
                arg_sw = i;
    
    return arg_b,arg_w,arg_sw

In [15]:
def centroid(s,argWorst):
    return np.mean(np.delete(s, argWorst), axis = 0)

In [16]:
def SimplexLocalSearch(option,x,Epsilon,Lambda,M,Alpha=1,Gamma=2,Beta=-0.5):
    N = len(x)
    s = np.empty([N+1,N])
    s[0] = x
    for j in range(1,N+1):
        s[j] = x + Lambda*(u[j-1]-l[j-1])*_e(j-1,N);
    k = 0
    
    
    argBest, argWorst, argSecondWorst = findExtremes(s)
    sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
    #set_trace()
    while abs(f(sBest) - f(sWorst)) > Epsilon and k < M and k < 1000:
        #set_trace()
        #if(k%1000 == 0):print(f(sBest),'\n')
        r = (1+Alpha)*centroid(s,argWorst) - Alpha*sWorst
        if(better(sBest,r) and better(r,sSecondWorst)):
            s[argWorst] = r
        elif(better(r,sBest)):
            e = (1+Gamma)*centroid(s,argWorst)-Gamma*sWorst
            if(better(e,r)):
                s[argWorst] = e
            else:
                s[argWorst] = r
        else:
            c = (1+Beta)*centroid(s,argWorst)-Beta*sWorst
            if(better(c,sWorst)):
                s[argWorst] = c
            else:
                if(option == 1): 
                    print(f(sBest))
                    return sBest, k, s, argBest, argWorst, argSecondWorst
                for j in range(0,N+1):
                    s[j] = (s[j] + sBest)/2

        #set_trace()
        argBest, argWorst, argSecondWorst = findExtremes(s)
        sBest, sWorst, sSecondWorst = s[argBest], s[argWorst], s[argSecondWorst]
        k = k + 1
    #print(f(sBest))
    return sBest, k, s, argBest, argWorst, argSecondWorst

In [17]:
import copy
def DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M, Gamma = 1.25,Beta=-0.5):
    k = 0
    it = 0
    x = np.random.uniform(low=lowerBound, high=upperBound, size=(10,))
    while(k < M):
        #set_trace()
        temp, _k, s, argBest, argWorst, argSecondWorst = SimplexLocalSearch(1,x,Epsilon,Lambda,M-k)
        x = copy.copy(temp)
        
        k = k + _k
        try: xBest
        except NameError: xBest = None
        
        if xBest is None or f(x) < f(xBest):
            temp, _k, s, argBest, argWorst, argSecondWorst = SimplexLocalSearch(2,x,EpsilonApostrophe,Lambda,M-k)
            xBest = np.zeros(N)
            xBest = xBest + temp
            k = k + _k
            print("***NEW BEST VALUE*** iteration: ", k, "| best found value: ", f(xBest))
            
        if xBest is not None and k//1000 > it:
            it = k//1000
            print("iteration:", it * 1000, "| best found value: ", f(xBest))
            
            #if f(xBest) < 1:
            #    for i in range(0,N):
            #        s[i][i] = s[i][i] + 10e-2*f(xBest)
            #else:
            #for i in range(0,N):
            #    s[i][i] = s[i][i] + 10e-15
            
        argBest, argWorst, argSecondWorst = findExtremes(s)
        
        
        
        while better(x,s[argWorst]):
            s[argWorst] = copy.copy(x)
            
            argBest, argWorst, argSecondWorst = findExtremes(s)
            x = Gamma*s[argBest] + (1-Gamma) * centroid(s,argBest)
            #if Delta(x)==0: print("+", k)
            s[argBest] = copy.copy(x)
            argBest, argWorst, argSecondWorst = findExtremes(s)

            
        
        
    return xBest

In [18]:
Lambda = 10
M = 100000
Epsilon = 0
#set_trace()
EpsilonApostrophe = 0
print(DirectionalEscape(Epsilon,EpsilonApostrophe,Lambda,M))

191493.30804802506
***NEW BEST VALUE*** iteration:  423 | best found value:  5384.487706144277
8435.48378827154
8921.488127937508
6927.9293726301175
3537.9943645538224
***NEW BEST VALUE*** iteration:  1366 | best found value:  1163.2002763269309
iteration: 1000 | best found value:  1163.2002763269309
1175.2743883445264
787.1210481278216
***NEW BEST VALUE*** iteration:  2142 | best found value:  0.17336815215518928
iteration: 2000 | best found value:  0.17336815215518928
0.17510830841375816
1.1630839339358147e-06
***NEW BEST VALUE*** iteration:  3673 | best found value:  3.18810065892556e-20
iteration: 3000 | best found value:  3.18810065892556e-20
1.1630839339358147e-06
7.503617336540584e-13
iteration: 4000 | best found value:  3.18810065892556e-20
8.197538247684311e-13
7.36341693890159e-18
iteration: 5000 | best found value:  3.18810065892556e-20
8.045346218011347e-18
1.2596603449834257e-08
0.00020901006856537227
0.03318899117263987
iteration: 6000 | best found value:  3.1881006589255

KeyboardInterrupt: 